In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime

now = datetime.today()
print("Starting pipeline execution at", now)

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
import pandera as pa
from IPython.display import HTML
import os

from avoidable_admissions.data.validate import (
    EmergencyCareEpisodeSchema,
    EmergencyCareFeatureSchema,
    get_schema_properties,
    validate_dataframe,
)
from avoidable_admissions.features import feature_maps
from avoidable_admissions.features.build_features import build_emergency_care_features

In [ ]:
from avoidable_admissions.data.validate import nhsdd_snomed

In [ ]:
path = "T:/Business Intelligence/Data Science/Work/hdruk_avoidable_adms/"

In [ ]:
df_edcare = pd.read_csv(os.path.join(path, "ed_care.csv"))

In [ ]:
episode_schema_properties = (
    get_schema_properties(EmergencyCareEpisodeSchema)
    .sort_values("name")
    .set_index("name")
)
episode_schema_properties.drop(columns=["title", "description"])

In [ ]:
dfe = df_edcare.copy()

dfe.townsend_score_quintile = dfe.townsend_score_quintile.fillna(0).astype('int64')
dfe.accommodationstatus = dfe.accommodationstatus.fillna(0)
dfe.edcomorb_01 = dfe.edcomorb_01.fillna(0).astype('int64')
dfe.edrefservice = dfe.edrefservice.fillna(0).astype('int64')
dfe.edwaittime = dfe.edwaittime.astype('float64')
dfe.timeined = dfe.timeined.astype('float64')
dfe.edarrivaldatetime = pd.to_datetime(dfe.edarrivaldatetime, yearfirst=True, utc = True)
dfe['visit_id'] = dfe.index

In [ ]:
dfe.edattendcat = dfe.edattendcat.astype(str)
dfe.eddepttype = dfe.eddepttype.astype(str)
dfe.gender = dfe.gender.astype(str)

dfe.accommodationstatus = dfe.accommodationstatus.astype('int64')
dfe.edacuity = dfe.edacuity.astype('int64')
dfe.edarrivalmode = dfe.edarrivalmode.astype('int64')
dfe.edrefservice = dfe.edrefservice.astype('int64')
dfe.eddiagqual_01 = 0

In [ ]:
dfe['eddiagqual_01'] = 415684004
dfe['accommodationstatus'] = 1066881000000100

In [ ]:
good, bad = validate_dataframe(
    dfe,
    EmergencyCareEpisodeSchema,
    #start_date=datetime(2021, 10, 1),
    #end_date=datetime(2022, 11, 1),
    ignore_cols=["eddiag_[0-9]{2}$","edinvest_[0-9]{2}$","eddiagqual_01"],
    update_cols={
        "edchiefcomplaint": {"checks": []},
    }
)

print("Good dataframe has %d rows" % good.shape[0])
print("Bad dataframe has %d rows" % bad.shape[0])

In [ ]:
dff = build_emergency_care_features(dfe.copy())

In [ ]:
good_f, bad_f = validate_dataframe(
    dff,
    EmergencyCareFeatureSchema,
    start_date=datetime(2021, 10, 1),
    end_date=datetime(2022, 11, 1),
    ignore_cols=["eddiag_[0-9]{2}$","edinvest_[0-9]{2}$","edwaittime","edarrivalmode_cat","edtreat_[0-9]{2}_cat","eddiag_seasonal_cat"],
    update_cols={
        "edchiefcomplaint": {"checks": []},
        "eddiag_01": {"checks": []},
        "eddiag_[0-9]{2}$": {"checks": []},
        "edinvest_[0-9]{2}$": {"checks": []},
        "edattendsource": {"checks": []},
        "edwaittime": {"checks": []},
    },
)
print("Good dataframe has %d rows" % good_f.shape[0])
print("Bad dataframe has %d rows" % bad_f.shape[0])

In [ ]:
df = good_f.copy()

In [ ]:
# All attendances - this should be redundant if done at extraction
df = df[(df.edattendcat == "1") & (df.eddepttype == "1")]
df.shape

In [ ]:
df["is_acsc"] = df.eddiag_01_acsc.str.startswith("ERROR")
df.is_acsc = df.is_acsc.replace({True: "Non-ACSC", False: "ACSC"})
df.is_acsc = np.where(df.eddiag_01_acsc == '-', 'Non-ACSC', df.is_acsc)

In [ ]:
df.townsend_score_quintile = np.where(df.townsend_score_quintile == 0, np.NaN, df.townsend_score_quintile)

#### ECDS (ED) Dataset Tables

In [ ]:
categorical_features = {
    "activage_cat": "Age Bands",
    "gender_cat": "Gender",
    "ethnos_cat": "Ethnicity",
    "townsend_score_quintile": "Townsend Score Quintile",
    "edarrivalmode_cat": "Arrival Mode",
    "edattendsource_cat": "Source of Attendance",
    "edarrival_dayofweek": "ED Arrival Day of Week",
    "edarrival_hourofday": "ED Arrival Hour of Day",
    "edacuity_cat": "Acuity",
    #"eddiag_seasonal_cat": "Seasonal Diagnosis",
    "edchiefcomplaint_cat": "Chief Complaint",
    "edinvest_01_cat": "Investigations",
    "edtreat_01_cat": "Treatment",
    "eddiagqual_01_cat": "Certainity",
    "edattenddispatch_cat": "Discharge Destination",
    "edrefservice_cat": "Service referred to",
}

In [ ]:
def make_crosstab(colname: str, tablename: str) -> pd.DataFrame:
    x = pd.crosstab(
        df_site[colname], df_site.is_acsc, margins=False, dropna=False, margins_name="Total"
    )

    y = (
        pd.crosstab(
            df_site[colname],
            df_site.is_acsc,
            normalize="index",
            dropna=False,
            margins_name="Total",
        )
        .mul(100)
        .round(2)
        .rename(columns={"ACSC": "ACSC %", "Non-ACSC": "Non-ACSC %"})
    )

    z = pd.concat([x, y], axis=1).sort_index(axis=1).fillna("-")

    #z['ACSC'].values[(z['ACSC'].values > 0) & (z['ACSC'].values < 10)] = 10
    #z['Non-ACSC'].values[z['Non-ACSC'].values < 10] = 10
    #z['ACSC %'] = z['ACSC']/(z['ACSC']+z['Non-ACSC'])
    #z['Non-ACSC %'] = z['Non-ACSC']/(z['ACSC']+z['Non-ACSC'])

    z.insert(loc=0, column='All Attendances', value=z['ACSC']+z['Non-ACSC'])
    z.insert(loc=1, column='All Attendances (%)', value=z['All Attendances']/sum(z['All Attendances']))
    z.insert(loc=0, column='Site', value=filter)
    z.index = pd.MultiIndex.from_tuples([(tablename, i) for i in z.index])
    

    return z

In [ ]:
def Q1(growth_vals:pd.Series):
    return growth_vals.quantile(.25)

def Q3(growth_vals:pd.Series):
    return growth_vals.quantile(.75)

cat_features = {
    "activage": "Age",
    "timeined": "Time in Department"
        }
    
def make_summary(colname, tablename):
    x = pd.DataFrame(pd.pivot_table(df_site, values=[k], columns=['is_acsc'],  
    aggfunc={np.size, min, max, np.mean, np.std, np.median, Q1, Q3})).reset_index().rename(columns={'level_0':'Group', 'level_1':'Measure', 'winter':'Oct-March', 'not-winter':'Apr-Sept'})
    y = pd.DataFrame(pd.pivot_table(df_site, values=[k], columns=['procodet'],  
    aggfunc={np.size, min, max, np.mean, np.std, np.median, Q1, Q3})).reset_index().rename(columns={'level_0':'Group', 'level_1':'Measure', 'winter':'Oct-March', 'not-winter':'Apr-Sept'})
    y = y.rename(columns={'RXN00': 'Total'})
    x = pd.concat([y[['Measure', 'Total']], x[['ACSC','Non-ACSC']]], axis=1)
    x.insert(loc=0, column='Group', value=k)
    x.insert(loc=0, column='Site', value=filter)
    return x

In [ ]:
# ANALYSIS TABLES

# Filter is for Site Split
filter = 'RXN01'
df_site = df[df['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    df_results.append(z)
df_results_s = pd.concat(df_results)

filter = 'RXN02'
df_site = df[df['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    df_results.append(z)
df_results = pd.concat(df_results)

df_results = pd.concat([df_results_s,df_results])
df_results.to_csv(os.path.join(path, "results/avoidable_admissions/", "ecds_analysis_tables.csv"))

# STATISTICAL SUMMARIES

filter = 'RXN01'
df_site = df[df['edsitecode'] == filter]

out = ""
df_results = []
for k, v in cat_features.items():
    z = make_summary(k, v)
    df_results.append(z)

df_results_s = pd.concat(df_results)
df_results_s.Measure = np.where(df_results_s.Measure == 'size', 'N', df_results_s.Measure) 

filter = 'RXN02'
df_site = df[df['edsitecode'] == filter]

out = ""
df_results = []
for k, v in cat_features.items():
    z = make_summary(k, v)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results.Measure = np.where(df_results.Measure == 'size', 'N', df_results.Measure) 
df_results = pd.concat([df_results_s,df_results])

df_results.to_csv(os.path.join(path, "results/avoidable_admissions/", "ecds_summary_statistics.csv"))



#### ACSC Analysis

In [ ]:
categorical_features = {
    "activage_cat": "Age Bands",
    "gender_cat": "Gender",
    "ethnos_cat": "Ethnicity",
    "townsend_score_quintile": "Townsend Score Quintile",
    "edarrivalmode_cat": "Arrival Mode",
    "edattendsource_cat": "Source of Attendance",
    "edarrival_dayofweek": "ED Arrival Day of Week",
    "edarrival_hourofday": "ED Arrival Hour of Day",
    "edacuity_cat": "Acuity",
    "edchiefcomplaint_cat": "Chief Complaint",
    "edinvest_01_cat": "Investigations",
    "edtreat_01_cat": "Treatment",
    "eddiagqual_01_cat": "Certainity",
    "edrefservice_cat": "Service referred to",
}

In [ ]:
def make_crosstab(colname: str, tablename: str) -> pd.DataFrame:
    x = pd.crosstab(
        df_site[colname], df_site.edattenddispatch_cat, margins=False, dropna=False, margins_name="Total"
    )

    y = (
        pd.crosstab(
            df_site[colname],
            df_site.edattenddispatch_cat,
            normalize="index",
            dropna=False,
            margins_name="Total",
        )
        .mul(100)
        .round(2)
        .rename(columns={"Admitted": "Admitted %", "Ambulatory / Short Stay": "Ambulatory/Short Stay %", "Died": "Died %", "Discharged": "Discharged %", "ERROR:Missing Data": "ERROR:Missing Data %", "Transfer": "Transfer %"})
    )

    z = pd.concat([x, y], axis=1).sort_index(axis=1).fillna("-")

    #z['ACSC'].values[(z['ACSC'].values > 0) & (z['ACSC'].values < 10)] = 10
    #z['Non-ACSC'].values[z['Non-ACSC'].values < 10] = 10
    #z['ACSC %'] = z['ACSC']/(z['ACSC']+z['Non-ACSC'])
    #z['Non-ACSC %'] = z['Non-ACSC']/(z['ACSC']+z['Non-ACSC'])

    #z.insert(loc=0, column='All Attendances', value=z['ACSC']+z['Non-ACSC'])
    #z.insert(loc=1, column='All Attendances (%)', value=z['All Attendances']/sum(z['All Attendances']))
    
    z.insert(loc=0, column='Site', value=filter)
    z.index = pd.MultiIndex.from_tuples([(tablename, i) for i in z.index])    

    return z

In [ ]:
def Q1(growth_vals:pd.Series):
    return growth_vals.quantile(.25)

def Q3(growth_vals:pd.Series):
    return growth_vals.quantile(.75)

cat_features = {
    "activage": "Age",
    "timeined": "Time in Department"
        }
    
def make_summary(colname, tablename):
    x = pd.DataFrame(pd.pivot_table(df_site, values=[k], columns=['edattenddispatch_cat'],  
    aggfunc={np.size, min, max, np.mean, np.std, np.median, Q1, Q3})).reset_index().rename(columns={'level_0':'Group', 'level_1':'Measure'})
    x.insert(loc=0, column='Site', value=filter)
    return x

In [ ]:
# ANALYSIS TABLES

filter = 'RXN01'
df_site = df[(df['edsitecode'] == filter) & (df.is_acsc == 'ACSC')]

df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    df_results.append(z)

df_results_s = pd.concat(df_results)

filter = 'RXN02'
df_site = df[(df['edsitecode'] == filter) & (df.is_acsc == 'ACSC')]

df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
df_results.to_csv(os.path.join(path, "results/avoidable_admissions/", "acsc_analysis_tables.csv"))

# SUMMARY STATISTICS

filter = 'RXN01'
df_site = df[(df['edsitecode'] == filter) & (df.is_acsc == 'ACSC')]

df_results = []
for k, v in cat_features.items():
    z = make_summary(k, v)
    df_results.append(z)

df_results_s = pd.concat(df_results)
df_results_s.Measure = np.where(df_results_s.Measure == 'size', 'N', df_results_s.Measure) 

filter = 'RXN02'
df_site = df[(df['edsitecode'] == filter) & (df.is_acsc == 'ACSC')]

df_results = []
for k, v in cat_features.items():
    z = make_summary(k, v)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results.Measure = np.where(df_results.Measure == 'size', 'N', df_results.Measure) 
df_results = pd.concat([df_results_s,df_results])
df_results.to_csv(os.path.join(path, "results/avoidable_admissions/", "acsc_summary_statistics.csv"), index=False)


# Winter Pressures

## Data Prep

In [ ]:
df_wp = good_f.copy()
#df_wp = df_wp[df_wp.townsend_score_quintile != 0]
df_wp.edarrivaldatetime =  pd.to_datetime(df_wp.edarrivaldatetime, format='%Y-%m-%d')

# Set deprivation as object
df_wp.townsend_score_quintile = df_wp.townsend_score_quintile.astype('object')

# Creat binary outcome variables
df_wp['over_1_invest'] = np.where(df_wp.edinvest_02_cat == 'No-investigation', '<=1', '>1')
df_wp['over_1_treat'] = np.where(df_wp.edtreat_02_cat == 'No-treatment', '<=1', '>1')

# Merge categories
df_wp['ethnos_cat'] = np.where(df_wp.ethnos_cat == 'Not stated', 'Not known', np.where(df_wp.ethnos_cat == 'Not known', 'Not known', np.where(df_wp.ethnos_cat == 'Other Ethnic Groups', 'Not known', df_wp.ethnos_cat)))
df_wp['eddiag_seasonal_cat'] = np.where(df_wp['eddiag_seasonal_cat'].str.contains('ERROR'),'None',df_wp.eddiag_seasonal_cat)
df_wp['edarrivalmode_group'] = np.where(df_wp['edarrivalmode_cat'] == 'Ambulance','Arrival: Ambulance','Arrival: Other')

# Update datetime
df_wp['edarrivaldate'] =  pd.to_datetime(df_wp['edarrivaldatetime']).dt.date
df_wp['edarrivalday'] =  pd.to_datetime(df_wp['edarrivaldate']).dt.day_name()
df_wp['edarrivaltime'] =  pd.to_datetime(df_wp['edarrivaldatetime']).dt.time.astype(str)

# Flag time of day
f = pd.Series(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
df_wp['time_of_day'] = np.where((df_wp.edarrivaltime.str[:2].astype('float') >= 8) & (df_wp.edarrivaltime.str[:2].astype('float') < 18) & (df_wp['edarrivalday'].isin(f)), 'in-hours', 'out-hours')

# All attendances - this should be redundant if done at extraction
df_wp = df_wp[(df_wp.edattendcat == "1") & (df_wp.eddepttype == "1")]

df_wp["is_acsc"] = df_wp.eddiag_01_acsc.str.startswith("ERROR")
df_wp.is_acsc = df_wp.is_acsc.replace({True: "Non-ACSC", False: "ACSC"})
df_wp.is_acsc = np.where(df_wp.eddiag_01_acsc == '-', 'Non-ACSC', df_wp.is_acsc)

df_wp.townsend_score_quintile = np.where(df_wp.townsend_score_quintile == 0, np.NaN, df_wp.townsend_score_quintile)

df_wp.shape

In [ ]:
#cols=['edinvest_[0-9]{2}_cat']
df_wp['urgent_treat'] = (df_wp.filter(regex="edtreat_[0-9]{2}_cat").astype(str) == 'Urgent').sum(axis=1)
df_wp['non_urgent_treat'] = (df_wp.filter(regex="edtreat_[0-9]{2}_cat").astype(str) == 'Non-urgent').sum(axis=1)

df_wp['urgent_invest'] = (df_wp.filter(regex="edinvest_[0-9]{2}_cat").astype(str) == 'Urgent').sum(axis=1)
df_wp['non_urgent_invest'] = (df_wp.filter(regex="edinvest_[0-9]{2}_cat").astype(str) == 'Non-urgent').sum(axis=1)

In [ ]:
# Create a series of boolean masks or filters based on the Sheffield specs
mask_eddeptype = df_wp.eddepttype == "1"
mask_edattendcat = df_wp.edattendcat == "1"
mask_disstatus_cat = df_wp.disstatus_cat == "Non-urgent"

# Boolean mask for Non-urgent investigations
mask_edinvest = (df_wp.filter(regex="edinvest_[0-9]{2}_cat") == "Non-urgent") | (df_wp.filter(regex="edinvest_[0-9]{2}_cat") == "ERROR:Missing Data") | (df_wp.filter(regex="edinvest_[0-9]{2}_cat") == "No-investigation")
mask_edinvest = mask_edinvest.all(axis=1)

# Boolean mask for Non-urgent treatments
mask_edtreat = (df_wp.filter(regex="edtreat_[0-9]{2}_cat") == "Non-urgent") | (df_wp.filter(regex="edtreat_[0-9]{2}_cat") == "ERROR:Missing Data") | (df_wp.filter(regex="edtreat_[0-9]{2}_cat") == "No-treatment")
mask_edtreat = mask_edtreat.all(axis=1)

# Boolean mask for Non-urgent attendances
mask_nonurgent = mask_eddeptype & mask_edattendcat & mask_disstatus_cat & mask_edinvest & mask_edtreat
df_wp['attend_urgency'] = np.where(mask_nonurgent == True,'Non-urgent', 'Urgent')

In [ ]:
# Create Winter categories
start_date, end_date = np.datetime64('2021-10-01'), np.datetime64('2022-04-01')
df_wp['winter_flag'] = np.where((df_wp.edarrivaldate >= start_date) & (df_wp.edarrivaldate < end_date), 'winter', 'not-winter')

start_date = np.datetime64('2022-10-01')
df_wp['winter_flag'] = np.where((df_wp.edarrivaldate >= start_date), 'winter', df_wp.winter_flag)

start_date, end_date = np.datetime64('2021-12-01'), np.datetime64('2022-03-01')
df_wp['winter_flag2'] = np.where((df_wp.edarrivaldate >= start_date) & (df_wp.edarrivaldate < end_date), 'winter', 'not-winter')

### Part 1

In [ ]:
def Q1(growth_vals:pd.Series):
    return growth_vals.quantile(.25)

def Q3(growth_vals:pd.Series):
    return growth_vals.quantile(.75)

In [ ]:
categorical_features = {
    "activage": "Age",
    "timeined": "Time in Department",
    "edwaittime": "ED Waiting Time",
    "non_urgent_invest": "Non-urgent investigations",
    "urgent_invest": "Urgent investigations",
    "non_urgent_treat": "Non-urgent treatments",
    "urgent_treat": "Urgent treatments",
}

In [ ]:
def make_summary(colname, tablename):
    x = pd.DataFrame(pd.pivot_table(df_site, values=[k], columns=['winter_flag'],  
    aggfunc={np.size, min, max, np.mean, np.std, np.median, Q1, Q3})).reset_index().rename(columns={'level_0':'Group', 'level_1':'Measure', 'winter':'Oct-March', 'not-winter':'Apr-Sept'})
    x.insert(loc=0, column='Site', value=filter)
    return x

def make_summary2(colname, tablename):
    x = pd.DataFrame(pd.pivot_table(df_site, values=[k], columns=['winter_flag2'],  
    aggfunc={np.size, min, max, np.mean, np.std, np.median, Q1, Q3})).reset_index().rename(columns={'level_0':'Group', 'level_1':'Measure', 'winter':'Dec-Feb', 'not-winter':'Mar-Nov'})
    x.insert(loc=0, column='Site', value=filter)
    return x

In [ ]:
filter = 'RXN01'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    df_results.append(z)
df_results_s = pd.concat(df_results)
df_results_s

df_results = []
for k, v in categorical_features.items():
    z = make_summary2(k, v)
    df_results.append(z)
df_results_s2 = pd.concat(df_results)
df_results_s3 = pd.concat([df_results_s[['Site','Group','Measure','Oct-March','Apr-Sept']],df_results_s2[['Dec-Feb','Mar-Nov']]], axis=1)

filter = 'RXN02'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    df_results.append(z)
df_results_s4 = pd.concat(df_results)

df_results = []
for k, v in categorical_features.items():
    z = make_summary2(k, v)
    df_results.append(z)
df_results_s5 = pd.concat(df_results)
df_results_s6 = pd.concat([df_results_s4[['Site','Group','Measure','Oct-March','Apr-Sept']],df_results_s5[['Dec-Feb','Mar-Nov']]], axis=1)

df_results = pd.concat([df_results_s3,df_results_s6])
df_results.Measure = np.where(df_results.Measure == 'size', 'N', df_results.Measure)
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_1/", "ed_summary_statistics.csv"), index=False)

In [ ]:
def make_crosstab(colname, tablename):
    x = pd.crosstab(df_site[k], df_site.winter_flag, margins=False, dropna=False).rename(columns={"winter": "Winter (6 months)", "not-winter": "Non-Winter (6 months)"})
    y = (
        pd.crosstab(
            df_site[k],
            df_site.winter_flag,
            normalize="index",
            dropna=False,
        )
        .mul(100)
        .round(2)
        .rename(columns={"winter": "Winter (6 months) %", "not-winter": "Non-Winter (6 months) %"})
    )
    a = pd.crosstab(df_site[k], df_site.winter_flag2, margins=False, dropna=False).rename(columns={"winter": "Winter (3 months)", "not-winter": "Non-Winter (3 months)"})
    b = (
        pd.crosstab(
            df_site[k],
            df_site.winter_flag2,
            normalize="index",
            dropna=False,
        )
        .mul(100)
        .round(2)
        .rename(columns={"winter": "Winter (3 months) %", "not-winter": "Non-Winter (3 months) %"})
    )

    z = pd.concat([x, y, a, b], axis=1).sort_index(axis=1).fillna("-")
    z = z[['Winter (6 months)', 'Winter (6 months) %', 'Non-Winter (6 months)', 'Non-Winter (6 months) %', 'Winter (3 months)', 'Winter (3 months) %', 'Non-Winter (3 months)', 'Non-Winter (3 months) %']]
    z.insert(loc=0, column='Site', value=filter)
    z.index = pd.MultiIndex.from_tuples([(v, i) for i in z.index])
    
    return z

In [ ]:
categorical_features = {
    "activage_cat": "Age Bands",
    "gender_cat": "Gender",
    "ethnos_cat": "Ethnicity",
    "townsend_score_quintile": "Townsend Score Quintile",
    "edarrivalmode_cat": "Arrival Mode",
    "edattendsource_cat": "Source of Attendance",
    "time_of_day": "Time",
    "edacuity_cat": "Acuity",
    "edchiefcomplaint_cat": 'Chief Complaint',
    "eddiag_seasonal_cat": "Seasonal Diagnosis",
    "edinvest_01_cat": "Investigations",
    "edtreat_01_cat": "Treatments",
    "eddiagqual_01_cat": "Certainity",
    "edattenddispatch_cat": "Discharge Destination",
    "attend_urgency": "Attend Urgency"
}

In [ ]:
filter = 'RXN01'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    df_results.append(z)

df_results_s = pd.concat(df_results)


filter = 'RXN02'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_1/", "ed_analysis_tables.csv"))

### Part 2

In [ ]:
categorical_features = {
    "activage": "edarrivalmode_group",
    "timeined": "edarrivalmode_group",
    "edwaittime": "edarrivalmode_group",    
    "non_urgent_invest": "edarrivalmode_group",
    "urgent_invest": "edarrivalmode_group",
    "non_urgent_treat": "edarrivalmode_group",
    "urgent_treat": "edarrivalmode_group",
}

In [ ]:
def make_summary(colname, tablename):
    t = pd.DataFrame(pd.pivot_table(df_site, values=k, index=v, aggfunc={np.size, min, max, np.mean, np.std, np.median, Q1, Q3}, columns=['winter_flag']).reset_index())
    t_cols = pd.DataFrame(t.columns)
    t_cols['x'], t_cols['y'] = zip(*t_cols[0])
    t_cols['name'] = t_cols['x'] + ' (' + t_cols['y'] + ')'
    t.columns = t_cols.name.tolist()
    t = t.rename(columns={t.columns[0]:'group'})
    t.insert(loc=0, column='period', value='6-month')
    t.insert(loc=1, column='site', value=filter)
    q = pd.DataFrame(pd.pivot_table(df_site, values=k, index=v, aggfunc={np.size, min, max, np.mean, np.std, np.median, Q1, Q3}, columns=['winter_flag2']).reset_index())
    q_cols = pd.DataFrame(q.columns)
    q_cols['x'], q_cols['y'] = zip(*q_cols[0])
    q_cols['name'] = q_cols['x'] + ' (' + q_cols['y'] + ')'
    q.columns = q_cols.name.tolist()
    q = q.rename(columns={q.columns[0]:'group'})
    q.insert(loc=0, column='period', value='3-month')
    q.insert(loc=1, column='site', value=filter)
    t = pd.concat([t,q])
    return t

In [ ]:
filter = 'RXN01'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    z.insert(loc=1, column='measure', value=k)
    df_results.append(z)

df_results_s = pd.concat(df_results)

filter = 'RXN02'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    z.insert(loc=1, column='measure', value=k)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
df_results = df_results.rename(columns={'size (not-winter)': 'N (not-winter)','size (winter)': 'N (winter)'})
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_2/", "ed_summary_statistics.csv"), index=False)

In [ ]:
def make_crosstab(colname, tablename):
    x = pd.crosstab(df_site[k], df_site.winter_flag, margins=False, dropna=False).rename(columns={"winter": "Winter (6 months)", "not-winter": "Non-Winter (6 months)"})
    y = (
        pd.crosstab(
            df_site[k],
            df_site.winter_flag,
            normalize="index",
            dropna=False,
        )
        .mul(100)
        .round(2)
        .rename(columns={"winter": "Winter (6 months) %", "not-winter": "Non-Winter (6 months) %"})
    )
    a = pd.crosstab(df_site[k], df_site.winter_flag2, margins=False, dropna=False).rename(columns={"winter": "Winter (3 months)", "not-winter": "Non-Winter (3 months)"})
    b = (
        pd.crosstab(
            df_site[k],
            df_site.winter_flag2,
            normalize="index",
            dropna=False,
        )
        .mul(100)
        .round(2)
        .rename(columns={"winter": "Winter (3 months) %", "not-winter": "Non-Winter (3 months) %"})
    )

    z = pd.concat([x, y, a, b], axis=1).sort_index(axis=1).fillna("-")
    z = z[['Winter (6 months)', 'Winter (6 months) %', 'Non-Winter (6 months)', 'Non-Winter (6 months) %', 'Winter (3 months)', 'Winter (3 months) %', 'Non-Winter (3 months)', 'Non-Winter (3 months) %']]
    z.insert(loc=0, column='Site', value=filter)
    z.index = pd.MultiIndex.from_tuples([(v, i) for i in z.index])
    
    return z

In [ ]:
categorical_features = {
    "activage_cat": "Age Bands",
    "gender_cat": "Gender",
    "ethnos_cat": "Ethnicity",
    "townsend_score_quintile": "Townsend Score Quintile",
    "edarrivalmode_cat": "Arrival Mode",
    "edattendsource_cat": "Source of Attendance",
    "time_of_day": "Time",
    "edacuity_cat": "Acuity",
    "edchiefcomplaint_cat": 'Chief Complaint',
    "eddiag_seasonal_cat": "Seasonal Diagnosis",
    "edinvest_01_cat": "Investigations",
    "edtreat_01_cat": "Treatments",
    "eddiagqual_01_cat": "Certainity",
    "edattenddispatch_cat": "Discharge Destination",
    "attend_urgency": "Attend Urgency"
}

In [ ]:
filter = 'RXN01'
strata = 'Arrival: Ambulance'

df_site = df_wp[(df_wp['edarrivalmode_group']==strata) & (df_wp['edsitecode'] == filter)]
out = ""
df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

strata = 'Arrival: Other'
df_site = df_wp[(df_wp['edarrivalmode_group']==strata) & (df_wp['edsitecode'] == filter)]
#out = ""
#df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

df_results_s = pd.concat(df_results)


filter = 'RXN02'
strata = 'Arrival: Ambulance'

df_site = df_wp[(df_wp['edarrivalmode_group']==strata) & (df_wp['edsitecode'] == filter)]
out = ""
df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

strata = 'Arrival: Other'
df_site = df_wp[(df_wp['edarrivalmode_group']==strata) & (df_wp['edsitecode'] == filter)]
#out = ""
#df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_2/", "ed_analysis_tables.csv"))

### Part 3

In [ ]:
categorical_features = {    
    "activage": "time_of_day",
    "timeined": "time_of_day",
    "edwaittime": "time_of_day",    
    "non_urgent_invest": "time_of_day",
    "urgent_invest": "time_of_day",
    "non_urgent_treat": "time_of_day",
    "urgent_treat": "time_of_day",
}

In [ ]:
filter = 'RXN01'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    z.insert(loc=1, column='measure', value=k)
    df_results.append(z)

df_results_s = pd.concat(df_results)

filter = 'RXN02'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    z.insert(loc=1, column='measure', value=k)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
df_results = df_results.rename(columns={'size (not-winter)': 'N (not-winter)','size (winter)': 'N (winter)'})
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_3/", "ed_summary_statistics.csv"), index=False)

In [ ]:
categorical_features = {
    "activage_cat": "Age Bands",
    "gender_cat": "Gender",
    "ethnos_cat": "Ethnicity",
    "townsend_score_quintile": "Townsend Score Quintile",
    "edarrivalmode_cat": "Arrival Mode",
    "edattendsource_cat": "Source of Attendance",
    "time_of_day": "Time",
    "edacuity_cat": "Acuity",
    "edchiefcomplaint_cat": 'Chief Complaint',
    "eddiag_seasonal_cat": "Seasonal Diagnosis",
    "edinvest_01_cat": "Investigations",
    "edtreat_01_cat": "Treatments",
    "eddiagqual_01_cat": "Certainity",
    "edattenddispatch_cat": "Discharge Destination",
    "attend_urgency": "Attend Urgency"
}

In [ ]:
filter = 'RXN01'
strata = 'in-hours'

df_site = df_wp[(df_wp['time_of_day']==strata) & (df_wp['edsitecode'] == filter)]
out = ""
df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

strata = 'out-hours'
df_site = df_wp[(df_wp['time_of_day']==strata) & (df_wp['edsitecode'] == filter)]
#out = ""
#df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

df_results_s = pd.concat(df_results)


filter = 'RXN02'
strata = 'in-hours'

df_site = df_wp[(df_wp['time_of_day']==strata) & (df_wp['edsitecode'] == filter)]
out = ""
df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

strata = 'out-hours'
df_site = df_wp[(df_wp['time_of_day']==strata) & (df_wp['edsitecode'] == filter)]
#out = ""
#df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_3/", "ed_analysis_tables.csv"))

### Part 4

In [ ]:
categorical_features = {
    "activage": "attend_urgency",
    "timeined": "attend_urgency",
    "edwaittime": "attend_urgency",    
    "non_urgent_invest": "attend_urgency",
    "urgent_invest": "attend_urgency",
    "non_urgent_treat": "attend_urgency",
    "urgent_treat": "attend_urgency",
}

In [ ]:
filter = 'RXN01'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    z.insert(loc=1, column='measure', value=k)
    df_results.append(z)

df_results_s = pd.concat(df_results)

filter = 'RXN02'
df_site = df_wp[df_wp['edsitecode'] == filter]

df_results = []
for k, v in categorical_features.items():
    z = make_summary(k, v)
    z.insert(loc=1, column='measure', value=k)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
df_results = df_results.rename(columns={'size (not-winter)': 'N (not-winter)','size (winter)': 'N (winter)'})
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_4/", "ed_summary_statistics.csv"), index=False)

In [ ]:
categorical_features = {
    "activage_cat": "Age Bands",
    "gender_cat": "Gender",
    "ethnos_cat": "Ethnicity",
    "townsend_score_quintile": "Townsend Score Quintile",
    "edarrivalmode_cat": "Arrival Mode",
    "edattendsource_cat": "Source of Attendance",
    "time_of_day": "Time",
    "edacuity_cat": "Acuity",
    "edchiefcomplaint_cat": 'Chief Complaint',
    "eddiag_seasonal_cat": "Seasonal Diagnosis",
    "edinvest_01_cat": "Investigations",
    "edtreat_01_cat": "Treatments",
    "eddiagqual_01_cat": "Certainity",
    "edattenddispatch_cat": "Discharge Destination",
    "attend_urgency": "Attend Urgency"
}

In [ ]:
filter = 'RXN01'
strata = 'Urgent'

df_site = df_wp[(df_wp['attend_urgency']==strata) & (df_wp['edsitecode'] == filter)]
out = ""
df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

strata = 'Non-urgent'
df_site = df_wp[(df_wp['attend_urgency']==strata) & (df_wp['edsitecode'] == filter)]
#out = ""
#df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

df_results_s = pd.concat(df_results)


filter = 'RXN02'
strata = 'Urgent'

df_site = df_wp[(df_wp['attend_urgency']==strata) & (df_wp['edsitecode'] == filter)]
out = ""
df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

strata = 'Non-urgent'
df_site = df_wp[(df_wp['attend_urgency']==strata) & (df_wp['edsitecode'] == filter)]
#out = ""
#df_results = []
for k, v in categorical_features.items():
    z = make_crosstab(k, v)
    z.insert(loc=0, column='Strata', value=strata)
    df_results.append(z)

df_results = pd.concat(df_results)
df_results = pd.concat([df_results_s,df_results])
#df_results
df_results.to_csv(os.path.join(path, "results/winter_pressures/part_4/", "ed_analysis_tables.csv"))

## Linear Regression

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
import re

import warnings
warnings.filterwarnings("ignore")

### 6 Month Analysis

In [ ]:
# Create 6-month Winter category
start_date, end_date = np.datetime64('2021-10-01'), np.datetime64('2022-04-01')
df_wp['winter_flag'] = np.where((df_wp.edarrivaldate >= start_date) & (df_wp.edarrivaldate < end_date), 'winter', 'not-winter')

start_date = np.datetime64('2022-10-01')
df_wp['winter_flag'] = np.where((df_wp.edarrivaldate >= start_date), 'winter', df_wp.winter_flag)

In [ ]:
filter = 'RXN01'
dfm = df_wp[['edwaittime','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''edwaittime ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_1_six = results[['Site','Variable', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['edwaittime','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''edwaittime ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_1_six_2 = results[['Site','Variable', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_1_six = pd.concat([results_1_six,results_1_six_2])

In [ ]:
filter = 'RXN01'
dfm = df_wp[['timeined','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''timeined ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_2_six_2 = results[['Site', 'Variable', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['timeined','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''timeined ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_2_six = results[['Site', 'Variable', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_2_six = pd.concat([results_2_six_2,results_2_six])

In [ ]:
filter = 'RXN01'
dfm = df_wp[['over_1_invest','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_invest,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_3_six_2 = results[['Site', 'Variable', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['over_1_invest','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_invest,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_3_six = results[['Site', 'Variable', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_3_six = pd.concat([results_3_six_2,results_3_six])


In [ ]:
filter = 'RXN01'
dfm = df_wp[['over_1_treat','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_treat,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_4_six_2 = results[['Site', 'Variable', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['over_1_treat','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_treat,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_4_six = results[['Site', 'Variable', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_4_six = pd.concat([results_4_six_2,results_4_six])

In [ ]:
filter = 'RXN01'
dfm = df_wp[['attend_urgency','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(attend_urgency,Treatment(reference="Urgent")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_5_six_2 = results[['Site', 'Variable', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['attend_urgency','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(attend_urgency,Treatment(reference="Urgent")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable = results.Variable.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_5_six = results[['Site', 'Variable', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_5_six = pd.concat([results_5_six_2,results_5_six])

### 3 Month Analysis

In [ ]:
# Create 3-month Winter category
start_date, end_date = np.datetime64('2021-12-01'), np.datetime64('2022-03-01')
df_wp['winter_flag'] = np.where((df_wp.edarrivaldate >= start_date) & (df_wp.edarrivaldate < end_date), 'winter', 'not-winter')

In [ ]:
filter = 'RXN01'
dfm = df_wp[['edwaittime','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''edwaittime ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_1_three_2 = results[['Site', 'Variable2', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['edwaittime','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''edwaittime ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_1_three = results[['Site', 'Variable2', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_1_three = pd.concat([results_1_three_2,results_1_three])

In [ ]:
filter = 'RXN01'
dfm = df_wp[['timeined','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''timeined ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_2_three_2 = results[['Site', 'Variable2', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['timeined','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''timeined ~   C(winter_flag, Treatment(reference="not-winter"))+
                            activage+
                            C(gender_cat, Treatment(reference="Male"))+
                            C(ethnos_cat, Treatment(reference="White"))+
                            C(townsend_score_quintile, Treatment(reference=1))+
                            C(eddiag_seasonal_cat, Treatment(reference='None'))+
                            time_of_day'''

model = smf.ols(formula = formula, data=dfm)
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_2_three = results[['Site', 'Variable2', 'coef', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_2_three = pd.concat([results_2_three_2,results_2_three])

In [ ]:
filter = 'RXN01'
dfm = df_wp[['over_1_invest','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_invest,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_3_three_2 = results[['Site', 'Variable2', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['over_1_invest','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_invest,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_3_three = results[['Site', 'Variable2', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_3_three = pd.concat([results_3_three_2,results_3_three])

In [ ]:
filter = 'RXN01'
dfm = df_wp[['over_1_treat','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_treat,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_4_three_2 = results[['Site', 'Variable2', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['over_1_treat','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(over_1_treat,Treatment(reference="<=1")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_4_three = results[['Site', 'Variable2', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_4_three = pd.concat([results_4_three,results_4_three_2])

In [ ]:
filter = 'RXN01'
dfm = df_wp[['attend_urgency','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(attend_urgency,Treatment(reference="Urgent")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_5_three_2 = results[['Site', 'Variable2', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]

filter = 'RXN02'
dfm = df_wp[['attend_urgency','winter_flag','activage','gender_cat','ethnos_cat','townsend_score_quintile','eddiag_seasonal_cat','time_of_day']][df_wp['edsitecode'] == filter].dropna()

formula = '''C(attend_urgency,Treatment(reference="Urgent")) ~ C(winter_flag, Treatment(reference="not-winter"))+
                                                                    activage+
                                                                    C(gender_cat, Treatment(reference="Male"))+
                                                                    C(ethnos_cat, Treatment(reference="White"))+
                                                                    C(townsend_score_quintile, Treatment(reference=1))+
                                                                    C(eddiag_seasonal_cat, Treatment(reference='None'))+
                                                                    time_of_day'''

model = smf.glm(formula = formula, data=dfm, family=sm.families.Binomial())
result = model.fit()

results = pd.DataFrame(result.summary().tables[1])[1:]
results = results.rename(columns={0: 'Variable2', 1: 'coef', 2: 'std_err', 3: 'z', 4: 'p_value', 5: 'Lower 95% CL',6: 'Upper 95% CL'})
results['OR'] = np.exp(results.coef.astype(str).astype(float))
results.Variable2 = results.Variable2.astype(str)
results.insert(loc=0, column='Site', value=filter)
results_5_three = results[['Site', 'Variable2', 'OR', 'Lower 95% CL', 'Upper 95% CL', 'p_value']]
results_5_three = pd.concat([results_5_three_2,results_5_three])

### Results

In [ ]:
results_1 = pd.concat([results_1_six, results_1_three], axis=1)
results_2 = pd.concat([results_2_six, results_2_three], axis=1)
results_3 = pd.concat([results_3_six, results_3_three], axis=1)
results_4 = pd.concat([results_4_six, results_4_three], axis=1)
results_5 = pd.concat([results_5_six, results_5_three], axis=1)

In [ ]:
def extractor(val):
    if val.startswith('C('):
        pattern1 = 'C?\(?([A-Za-z]+_[A-Za-z]+_?[A-Za-z]+)'        
        pattern2 = '(\[.*\])'        
        
        match1 = re.findall(pattern1, val)[0]        
        match2 = re.findall(pattern2, val)[0]
        
        final = match1+match2        
        
        return final    
    else:
        return val
                
results_1['Variable'] = results_1['Variable'].apply(lambda x: extractor(x))
results_2['Variable'] = results_2['Variable'].apply(lambda x: extractor(x))
results_3['Variable'] = results_3['Variable'].apply(lambda x: extractor(x))
results_4['Variable'] = results_4['Variable'].apply(lambda x: extractor(x))
results_5['Variable'] = results_5['Variable'].apply(lambda x: extractor(x))

results_1['Variable2'] = results_1['Variable2'].apply(lambda x: extractor(x))
results_2['Variable2'] = results_2['Variable2'].apply(lambda x: extractor(x))
results_3['Variable2'] = results_3['Variable2'].apply(lambda x: extractor(x))
results_4['Variable2'] = results_4['Variable2'].apply(lambda x: extractor(x))
results_5['Variable2'] = results_5['Variable2'].apply(lambda x: extractor(x))

#### Create tables and save to CSV

In [ ]:
string_2 = ['','', '6-month winter analysis', '6-month winter analysis', '6-month winter analysis', '6-month winter analysis','','', '3-month winter analysis', '3-month winter analysis', '3-month winter analysis', '3-month winter analysis']
string_3 = ['Site', 'Variable','Coef','Lower 95% CL', 'Upper 95% CL','p_value','Site', 'Variable','Coef','Lower 95% CL', 'Upper 95% CL','p_value']

In [ ]:
cols = list(zip(string_2, string_3))
results_1.columns = pd.MultiIndex.from_tuples(cols)
results_1.to_csv(os.path.join(path, "results/winter_pressures/part_5/", "ed_waiting_time_regression.csv"), index=False)

In [ ]:
cols = list(zip(string_2, string_3))
results_2.columns = pd.MultiIndex.from_tuples(cols)
results_2.to_csv(os.path.join(path, "results/winter_pressures/part_5/", "ed_total_time_regression.csv"), index=False)

In [ ]:
#string_2 = ['', '6-month winter analysis', '6-month winter analysis', '6-month winter analysis', '6-month winter analysis', '3-month winter analysis', '3-month winter analysis', '3-month winter analysis', '3-month winter analysis']
string_3 = ['Site', 'Variable','OR','Lower 95% CL', 'Upper 95% CL','p_value','Site', 'Variable','OR','Lower 95% CL', 'Upper 95% CL','p_value']

In [ ]:
cols = list(zip(string_2, string_3))
results_3.columns = pd.MultiIndex.from_tuples(cols)
results_3.to_csv(os.path.join(path, "results/winter_pressures/part_5/", "ed_investigations_regression.csv"), index=False)

In [ ]:
cols = list(zip(string_2, string_3))
results_4.columns = pd.MultiIndex.from_tuples(cols)
results_4.to_csv(os.path.join(path, "results/winter_pressures/part_5/", "ed_treatments_regression.csv"), index=False)

In [ ]:
cols = list(zip(string_2, string_3))
results_5.columns = pd.MultiIndex.from_tuples(cols)
results_5.to_csv(os.path.join(path, "results/winter_pressures/part_5/", "ed_urgency_regression.csv"), index=False)